In [8]:
import os
import torch
from torch.utils.data import DataLoader, TensorDataset
from transformers import BertTokenizer, BertForSequenceClassification,BertModel, AdamW
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import tensorflow as tf
import pandas as pd
import numpy as np

EPOCHS = 5
BATCH_SIZE = 8
VALIDATION_SIZE = 0.2

CLASSES = {0: 'entailment', 1: 'neutral', 2: 'contradiction'}

In [2]:
train_df = pd.read_csv('data/train.csv')
test_df = pd.read_csv('data/test.csv')

In [12]:
#Convert DataFrame to list of pairs and labels
train_data = {(row['premise'], row['hypothesis']) : row['label'] for _, row in train_df.iterrows()}


In [4]:
#Tokenization
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
tokenized_train = tokenizer(train_data, padding='max_length', truncation=True, max_length=300, return_tensors='pt')

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(output_dir="test_trainer")

In [6]:
#Dataset and DataLoader
dataset = TensorDataset(tokenized_train['input_ids'], tokenized_train['attention_mask'], torch.tensor(labels))
dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)

In [7]:
#BERT Model
model = BertForSequenceClassification.from_pretrained('bert-base-multilingual-cased', num_labels=len(CLASSES))
optimizer = AdamW(model.parameters(), lr=2e-5)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
c:\Users\vlns\Python\Python311\Lib\site-packages\transformers\optimization.py:457: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [11]:
# Number of training epochs
from transformers import get_linear_schedule_with_warmup

EPOCHS = 3
total_steps = len(dataloader) * EPOCHS

# Learning rate scheduler
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)

# Training loop
model.train()
for epoch in range(EPOCHS):
    total_loss = 0
    for step, batch in enumerate(dataloader):
        # Unpack batch
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        labels = batch['labels']

        # Forward pass
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        total_loss += loss.item()

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)  # Gradient clipping to prevent exploding gradients
        optimizer.step()
        scheduler.step()

    # Calculate average loss for the epoch
    avg_train_loss = total_loss / len(dataloader)

    print(f"Epoch {epoch+1}/{EPOCHS} - Avg Training Loss: {avg_train_loss:.4f}")

print("Training complete!")

TypeError: list indices must be integers or slices, not str

In [9]:
#Training Loop
model.train()
for epoch in range(EPOCHS):
    for batch in dataloader:
        input_ids, attention_mask, labels = batch
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

KeyboardInterrupt: 

In [ ]:
# Save model checkpoint
torch.save({
    'epoch': epoch,
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    'loss': loss,
    # Add any other information you want to save
}, 'bert_model_checkpoint.pth')

In [ ]:
#TODO test_dataloader

In [ ]:
spredictions = []
# Iterate through the test data
for batch in test_dataloader:  # Assuming test_dataloader is similar to the training dataloader
    input_ids, attention_mask, labels = batch
    #Disable gradient calculation to speed up inference
    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
    # Get logits from the model's output
    logits = outputs.logits
    # Convert logits to probabilities if needed (e.g., for classification tasks)
    probabilities = torch.nn.functional.softmax(logits, dim=-1)
    # Get predicted labels
    predicted_labels = torch.argmax(logits, dim=-1)
    # Store predictions and true labels
    predictions.extend(predicted_labels.tolist())

# Convert predictions and true labels to numpy arrays for further analysis
predictions = np.array(predictions)